In [ ]:
import os
import time
from datetime import datetime

# 기존 시스템 함수 import (정확한 함수명 사용)
from src.config import (CONFIG, UNIFIED_CITY_INFO, is_url_processed_fast,
                       mark_url_processed_fast)
from src.utils.file_handler import (
    create_product_data_structure,
    save_to_csv_klook,
    get_dual_image_urls_klook,
    download_dual_images_klook,
    get_next_product_number,
    auto_create_country_csv_after_crawling,
    get_csv_stats
)
from src.scraper.parsers import extract_all_product_data
from src.scraper.driver_manager import setup_driver

# 크롤링 설정
CITY_NAME = "도쿄"  # 수정 필요시 변경
BATCH_SIZE = 50     # 진행률 표시 간격
REQUEST_DELAY = 1   # 요청 간 대기시간(초)

print(f"KLOOK Sitemap 크롤러 준비")
print(f"   대상 도시: {CITY_NAME}")
print(f"   시작 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*60)

In [ ]:
# Cell 2: URL 파일 처리 함수들

def load_sitemap_urls(city_name):
    """저장된 sitemap URL 파일에서 URL 목록 로드"""
    filename = f"sitemap_urls/{city_name}_urls.txt"

    if not os.path.exists(filename):
        print(f"URL 파일을 찾을 수 없습니다: {filename}")
        print("해결 방법:")
        print("   1. KLOOK_Sitemap_Collector.ipynb를 먼저 실행하세요")
        print("   2. 도시명이 정확한지 확인하세요")
        return []

    urls = []
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            for line_num, line in enumerate(f, 1):
                url = line.strip()
                if url:
                    if url.startswith('https://www.klook.com/activity/'):
                        urls.append(url)
                    else:
                        print(f"잘못된 URL 형식 (라인 {line_num}): {url}")

        print(f"URL 파일 로드 성공:")
        print(f"   파일: {filename}")
        print(f"   총 URL: {len(urls):,}개")
        return urls

    except Exception as e:
        print(f"URL 파일 읽기 실패: {e}")
        return []

def filter_new_urls_only(urls, city_name):
    """중복 체크하여 신규 URL만 반환 (기존 페이지네이션 수집 데이터와 비교)"""
    if not urls:
        return []

    print(f"중복 검사 시작: {len(urls):,}개 URL")

    new_urls = []
    duplicate_count = 0

    for i, url in enumerate(urls, 1):
        # 진행률 표시 (1000개마다)
        if i % 1000 == 0:
            progress = (i / len(urls)) * 100
            print(f"   진행: {i:,}/{len(urls):,} ({progress:.1f}%) - 신규: {len(new_urls):,}개")

        if not is_url_processed_fast(url, city_name):
            new_urls.append(url)
        else:
            duplicate_count += 1

    print(f"중복 검사 완료:")
    print(f"   신규 URL: {len(new_urls):,}개")
    print(f"   중복 제외: {duplicate_count:,}개")
    print(f"   신규 비율: {len(new_urls)/len(urls)*100:.1f}%")

    return new_urls

def validate_sitemap_setup(city_name):
    """크롤링 시작 전 환경 검증"""
    issues = []

    # URL 파일 존재 확인
    url_file = f"sitemap_urls/{city_name}_urls.txt"
    if not os.path.exists(url_file):
        issues.append(f"URL 파일 없음: {url_file}")

    # 디렉토리 구조 확인
    required_dirs = ["hash_index", "data", "klook_img"]
    for dir_name in required_dirs:
        if not os.path.exists(dir_name):
            issues.append(f"필수 디렉토리 없음: {dir_name}")

    # 도시 정보 확인
    if city_name not in UNIFIED_CITY_INFO:
        issues.append(f"지원되지 않는 도시: {city_name}")

    if issues:
        print("환경 검증 실패:")
        for issue in issues:
            print(f"   • {issue}")
        return False
    else:
        print("환경 검증 통과")
        return True

print("URL 파일 처리 함수 준비 완료")

In [ ]:
# Cell 3: 크롤링 실행 함수들

def create_sitemap_product_data(city_name, product_number, rank=None):
    """Sitemap 방식임을 표시하는 데이터 구조 생성"""
    data = create_product_data_structure(city_name, product_number, rank)
    data['수집방식'] = 'sitemap'  # 구분 컬럼 추가
    return data

def crawl_single_sitemap_url(driver, url, city_name, product_number, current, total):
    """단일 URL 크롤링 및 데이터 저장 (연속성 유지)"""
    try:
        print(f"[{current:,}/{total:,}] #{product_number} 크롤링...")
        print(f"   URL: {url}")

        # 페이지 방문
        driver.get(url)
        time.sleep(2)

        # 기본 데이터 구조 (sitemap 표시 포함)
        product_data = create_sitemap_product_data(city_name, product_number, current)
        product_data['URL'] = url

        # 상품 데이터 추출 (정확한 함수명 사용)
        extracted_data = extract_all_product_data(driver, url)

        if extracted_data:
            # 데이터 병합
            product_data.update(extracted_data)

            # 기존 CSV에 추가 저장 (연속성 유지)
            if save_to_csv_klook(product_data, city_name):
                print(f"   상품 #{product_number} CSV 저장 완료")

                # 이미지 다운로드 (연속성 유지)
                try:
                    image_urls = get_dual_image_urls_klook(driver)
                    if image_urls:
                        download_dual_images_klook(image_urls, product_number, city_name)
                        print(f"   이미지 다운로드 완료")
                except Exception as img_e:
                    print(f"   이미지 다운로드 실패: {img_e}")

                # 처리 완료 표시
                mark_url_processed_fast(url, city_name, product_number, current)

                return True, product_data.get('상품명', 'Unknown')
        else:
            print(f"   상품 데이터 추출 실패")
            return False, None

    except Exception as e:
        print(f"   크롤링 실패: {e}")
        return False, None

def safe_crawl_with_retry(driver, url, city_name, product_number, current, total, max_retries=2):
    """재시도 로직이 있는 안전한 크롤링"""
    for attempt in range(max_retries + 1):
        try:
            if attempt > 0:
                print(f"   재시도 {attempt}/{max_retries}")
                time.sleep(3)  # 재시도시 더 긴 대기

            success, product_name = crawl_single_sitemap_url(
                driver, url, city_name, product_number, current, total
            )

            if success:
                return True, product_name

        except Exception as e:
            if attempt == max_retries:
                print(f"   최종 실패: {e}")
                return False, None
            else:
                print(f"   시도 {attempt + 1} 실패: {e}")

    return False, None

print("크롤링 실행 함수 준비 완료")

In [ ]:
# Cell 4: 진행률 및 통계 함수들

def show_progress_summary(current, total, success_count, failed_urls, start_time, city_name):
    """상세한 진행률 및 통계 표시"""
    progress = (current / total) * 100
    elapsed = time.time() - start_time
    avg_time = elapsed / current if current > 0 else 0
    remaining = (total - current) * avg_time

    print(f"\n진행 상황 ({datetime.now().strftime('%H:%M:%S')})")
    print(f"   도시: {city_name}")
    print(f"   진행률: {current:,}/{total:,} ({progress:.1f}%)")
    print(f"   성공: {success_count:,}개")
    print(f"   실패: {len(failed_urls):,}개")
    print(f"   소요시간: {elapsed/60:.1f}분")
    print(f"   예상 잔여: {remaining/60:.1f}분")
    if current > 0:
        print(f"   성공률: {success_count/current*100:.1f}%")
        print(f"   평균 속도: {avg_time:.1f}초/URL")

def save_crawling_log(city_name, total_urls, success_count, failed_urls, start_time, end_time):
    """크롤링 결과 로그 저장"""
    log_data = {
        "도시": city_name,
        "시작시간": datetime.fromtimestamp(start_time).strftime('%Y-%m-%d %H:%M:%S'),
        "종료시간": datetime.fromtimestamp(end_time).strftime('%Y-%m-%d %H:%M:%S'),
        "소요시간": f"{(end_time - start_time)/60:.1f}분",
        "총_URL": total_urls,
        "성공": success_count,
        "실패": len(failed_urls),
        "성공률": f"{success_count/total_urls*100:.1f}%",
        "수집방식": "sitemap"
    }

    # 로그 파일 저장
    log_filename = f"crawling_logs/{city_name}_sitemap_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
    os.makedirs("crawling_logs", exist_ok=True)

    with open(log_filename, 'w', encoding='utf-8') as f:
        for key, value in log_data.items():
            f.write(f"{key}: {value}\n")

        if failed_urls:
            f.write(f"\n실패한 URL 목록:\n")
            for url in failed_urls:
                f.write(f"  - {url}\n")

    print(f"크롤링 로그 저장: {log_filename}")
    return log_data

print("진행률 및 통계 함수 준비 완료")

In [ ]:
# Cell 5: 메인 크롤링 실행

def main_sitemap_crawling(city_name):
    """Sitemap 기반 크롤링 메인 함수"""
    start_time = time.time()
    driver = None
    success_count = 0
    failed_urls = []
    successful_products = []

    try:
        print(f"\n'{city_name}' Sitemap 크롤링 시작")
        print("="*60)

        # 1. 환경 검증
        if not validate_sitemap_setup(city_name):
            return

        # 2. 저장된 URL 불러오기
        print(f"\n1단계: URL 파일 로드")
        sitemap_urls = load_sitemap_urls(city_name)
        if not sitemap_urls:
            return

        # 3. 기존 수집 데이터와 중복 체크하여 제외
        print(f"\n2단계: 중복 검사")
        new_urls = filter_new_urls_only(sitemap_urls, city_name)

        if not new_urls:
            print("\n모든 URL이 이미 처리되었습니다!")
            print("새로운 URL을 수집하려면 KLOOK_Sitemap_Collector.ipynb를 다시 실행하세요.")
            return

        # 4. 드라이버 설정
        print(f"\n3단계: 브라우저 드라이버 설정")
        driver = setup_driver(city_name)
        print(f"드라이버 준비 완료")

        # 5. 신규 URL 순차 크롤링 (연속성 유지)
        print(f"\n4단계: 크롤링 실행")
        print(f"   처리 대상: {len(new_urls):,}개 신규 URL")
        print(f"   예상 소요: {len(new_urls) * REQUEST_DELAY / 60:.1f}분")

        for i, url in enumerate(new_urls, 1):
            # 번호 연속성 유지 (기존 CSV 번호 이어서)
            product_number = get_next_product_number(city_name)

            # 안전한 크롤링 (재시도 포함)
            success, product_name = safe_crawl_with_retry(
                driver, url, city_name, product_number, i, len(new_urls)
            )

            if success:
                success_count += 1
                successful_products.append({
                    'number': product_number,
                    'name': product_name,
                    'url': url
                })
            else:
                failed_urls.append(url)

            # 진행률 표시
            if i % BATCH_SIZE == 0 or i == len(new_urls):
                show_progress_summary(i, len(new_urls), success_count, failed_urls, start_time, city_name)

            # 요청 간격 (서버 부하 방지)
            if i < len(new_urls):  # 마지막이 아닐때만
                time.sleep(REQUEST_DELAY)

        # 6. 최종 결과
        end_time = time.time()
        print(f"\n'{city_name}' Sitemap 크롤링 완료!")
        print("="*60)
        print(f"최종 통계:")
        print(f"   총 소요시간: {(end_time - start_time)/60:.1f}분")
        print(f"   처리 URL: {len(new_urls):,}개")
        print(f"   성공: {success_count:,}개")
        print(f"   실패: {len(failed_urls):,}개")
        print(f"   성공률: {success_count/len(new_urls)*100:.1f}%")

        if successful_products:
            print(f"\n수집 성공한 상품 (최근 5개):")
            for product in successful_products[-5:]:
                print(f"   #{product['number']}: {product['name']}")

        # 7. 크롤링 로그 저장
        save_crawling_log(city_name, len(new_urls), success_count, failed_urls, start_time, end_time)

        # 8. 국가별 통합 CSV 자동 생성
        if success_count > 0:
            print(f"\n5단계: 국가별 통합 CSV 생성")
            auto_create_country_csv_after_crawling(city_name)

    except KeyboardInterrupt:
        print(f"\n사용자에 의해 중단됨 (현재까지 {success_count}개 성공)")

    except Exception as e:
        print(f"\n크롤링 중 치명적 오류: {e}")

    finally:
        # 드라이버 정리 (안전한 종료)
        if driver:
            print(f"\n브라우저 드라이버 종료 중...")
            try:
                driver.quit()
                print(f"드라이버 종료 완료")
            except Exception as e:
                print(f"드라이버 종료 중 오류: {e}")

# 실행
print(f"크롤링 시작 준비 완료")
main_sitemap_crawling(CITY_NAME)

In [ ]:
# Cell 6: 결과 확인 및 통계

def show_final_statistics(city_name):
    """최종 수집 데이터 통계 표시"""
    print(f"\n'{city_name}' 최종 데이터 현황")
    print("="*60)

    # CSV 통계
    csv_stats = get_csv_stats(city_name)
    if csv_stats and 'error' not in csv_stats:
        print(f"CSV 파일 정보:")
        print(f"   파일: {csv_stats['file_path']}")
        print(f"   총 상품: {csv_stats['total_products']:,}개")
        print(f"   고유 상품: {csv_stats['unique_hashes']:,}개")
        print(f"   파일 크기: {csv_stats['file_size']:,} bytes ({csv_stats['file_size']/1024/1024:.1f}MB)")

        # sitemap 방식으로 수집된 데이터 확인
        try:
            import csv
            sitemap_count = 0
            pagination_count = 0

            with open(csv_stats['file_path'], 'r', encoding='utf-8-sig') as f:
                reader = csv.DictReader(f)
                for row in reader:
                    collection_method = row.get('수집방식', 'pagination')
                    if collection_method == 'sitemap':
                        sitemap_count += 1
                    else:
                        pagination_count += 1

            print(f"\n수집 방식별 통계:")
            print(f"   페이지네이션: {pagination_count:,}개")
            print(f"   Sitemap: {sitemap_count:,}개")

        except Exception as e:
            print(f"   방식별 통계 확인 실패: {e}")
    else:
        print(f"CSV 파일 통계 확인 실패")

    # 이미지 파일 확인
    from src.utils.file_handler import get_image_stats
    img_stats = get_image_stats(city_name)
    if img_stats and 'error' not in img_stats:
        print(f"\n이미지 파일 정보:")
        print(f"   디렉토리: {img_stats['directory']}")
        print(f"   총 이미지: {img_stats['total_images']:,}개")
        print(f"   총 크기: {img_stats['total_size']/1024/1024:.1f}MB")

def show_next_steps():
    """다음 단계 안내"""
    print(f"\n다음 단계 안내")
    print("="*60)
    print("완료된 작업:")
    print("   • Sitemap URL 수집 완료")
    print("   • 중복 검사 및 신규 URL 크롤링 완료")
    print("   • 기존 CSV에 데이터 추가 (연속성 유지)")
    print("   • 이미지 다운로드 완료")
    print("   • 국가별 통합 CSV 생성 완료")

    print(f"\n추가 가능한 작업:")
    print("   1. 다른 도시 크롤링:")
    print("      → Cell 1에서 CITY_NAME 변경 후 재실행")
    print("   2. 정기 업데이트:")
    print("      → KLOOK_Sitemap_Collector.ipynb → 이 노트북 순서로 실행")
    print("   3. 데이터 품질 검증:")
    print("      → 수집된 데이터의 완성도 확인")

# 최종 결과 확인 실행
show_final_statistics(CITY_NAME)
show_next_steps()

print(f"\nKLOOK Sitemap 크롤링 시스템 완료")
print(f"완료 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")